In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_json('loan_data.json')

In [3]:
df.sample(5)

,Application_ID,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status
361,LP002370,Male,No,0,Not Graduate,No,1,Urban,low,Y
297,LP002151,Male,Yes,1,Graduate,No,1,Urban,low,N
98,LP001401,Male,Yes,1,Graduate,No,1,Rural,high,Y
221,LP001868,Male,No,0,Graduate,No,1,Semiurban,low,Y
0,LP001002,Male,No,0,Graduate,No,1,Urban,medium,Y


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Application_ID      511 non-null    object
 1   Gender              511 non-null    object
 2   Married             511 non-null    object
 3   Dependents          511 non-null    object
 4   Education           511 non-null    object
 5   Self_Employed       511 non-null    object
 6   Credit_History      511 non-null    int64 
 7   Property_Area       511 non-null    object
 8   Income              511 non-null    object
 9   Application_Status  511 non-null    object
dtypes: int64(1), object(9)
memory usage: 40.0+ KB


# 1. Find % of total applicants for each unique value of dependents.

In [5]:
np.round((df['Dependents'].value_counts()/df.shape[0])*100,2)

0     57.53
2     17.22
1     16.63
3+     8.61
Name: Dependents, dtype: float64

# 2.Find the %of applications approved for self-employed applicants.

In [6]:
np.round((df[(df['Self_Employed']=='Yes') &(df['Application_Status']=='Y')].shape[0]/df[df['Self_Employed']=='Yes'].shape[0])*100,2)

65.71

# 3.What is the % of rejections for married male applicants?

In [7]:
np.round((df[(df['Gender']=='Male')&(df['Married']=='Yes')&(df['Application_Status']=='N')].shape[0]/df[(df['Gender']=='Male')&(df['Married']=='Yes')].shape[0])*100,2)

28.43

#  4.Which property area has the maximum approval ratio.

In [8]:
#property areas are
df['Property_Area'].unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

In [9]:
#Urban approved percentage
df[(df['Property_Area']=='Urban') & (df['Application_Status']=='Y')].shape[0]/df[(df['Property_Area']=='Urban')].shape[0]*100

63.030303030303024

In [10]:
#Rural approved percentage
df[(df['Property_Area']=='Rural') & (df['Application_Status']=='Y')].shape[0]/df[(df['Property_Area']=='Rural')].shape[0]*100

60.40268456375839

In [11]:
#semiurban approved percentage
df[(df['Property_Area']=='Semiurban') & (df['Application_Status']=='Y')].shape[0]/df[(df['Property_Area']=='Semiurban')].shape[0]*100

77.66497461928934

conclusion: Semiurban area has maximum aproval ratio

# 5.Find average number of dependents per income group.

In [12]:
#replce 3+ as 3 and then convert to string to numeric
df['Dependents']=df['Dependents'].replace(to_replace=['0','1','2','3+'],value=['0','1','2','3'])
df['Dependents']=pd.to_numeric(df['Dependents'])

In [13]:
df1=df.groupby('Income')['Dependents'].mean().reset_index()
df1.columns=['Income','Avg of Dependents']
df1

,Income,Avg of Dependents
0,high,0.888889
1,low,0.641026
2,medium,0.922280


# 6.Find approval ratio for various combinations of Property_Area and Marrital status

In [14]:
#Appoverd count for various combinations of Property_Area and Marrital status
df2=df[df['Application_Status']=='Y'][['Property_Area','Married','Application_Status']]
df2=df2.groupby(['Property_Area','Married']).count().reset_index()
df2

,Property_Area,Married,Application_Status
0,Rural,No,34
1,Rural,Yes,56
2,Semiurban,No,47
3,Semiurban,Yes,106
4,Urban,No,30
5,Urban,Yes,74


In [15]:
#No of applicants for various combinations of Property_Area and Marrital status
df3=df.groupby(['Property_Area','Married']).count()['Application_Status'].reset_index()
df3.columns=['Property_Area','Married','no of applicants']
df3

,Property_Area,Married,no of applicants
0,Rural,No,56
1,Rural,Yes,93
2,Semiurban,No,68
3,Semiurban,Yes,129
4,Urban,No,56
5,Urban,Yes,109


In [16]:
#Approval ratio for various combinations of Property_Area and Marrital status
df3['Approval_ratio(%)']=df2['Application_Status']/df3['no of applicants']*100
df3

,Property_Area,Married,no of applicants,Approval_ratio(%)
0,Rural,No,56,60.714286
1,Rural,Yes,93,60.215054
2,Semiurban,No,68,69.117647
3,Semiurban,Yes,129,82.170543
4,Urban,No,56,53.571429
5,Urban,Yes,109,67.889908


# 7. Create a simple predictive model to assess whether a loan application will be approved or rejected and provide the accuracy score.

In [17]:
#convert categorical into numerical
df['Gender']=df['Gender'].map({'Male':1,'Female':0})
df['Married']=df['Married'].map({'Yes':1,'No':0})
df['Education']=df['Education'].map({'Graduate':1,'Not Graduate':0})
df['Self_Employed']=df['Self_Employed'].map({'Yes':1,'No':0})
df['Property_Area']=df['Property_Area'].map({'Urban':1,'Rural':0,'Semiurban':2})
df['Income']=df['Income'].map({'high':1,'low':0,'medium':2})
df['Application_Status']=df['Application_Status'].map({'Y':1,'N':0})

In [18]:
df.corr()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status
Gender,1.000000,0.363499,0.200692,-0.044667,-0.007948,0.024682,-0.102943,0.031240,0.052533
Married,0.363499,1.000000,0.375597,0.002516,0.019750,0.020519,0.024597,-0.013830,0.098560
Dependents,0.200692,0.375597,1.000000,-0.039250,0.051180,-0.039492,-0.023291,0.131527,0.016984
Education,-0.044667,0.002516,-0.039250,1.000000,0.014796,0.075720,0.071831,0.145458,0.088699
Self_Employed,-0.007948,0.019750,0.051180,0.014796,1.000000,-0.016306,-0.024893,0.162880,-0.018705
Credit_History,0.024682,0.020519,-0.039492,0.075720,-0.016306,1.000000,0.056093,0.025586,0.545934
Property_Area,-0.102943,0.024597,-0.023291,0.071831,-0.024893,0.056093,1.000000,-0.029186,0.155912
Income,0.031240,-0.013830,0.131527,0.145458,0.162880,0.025586,-0.029186,1.000000,-0.003004
Application_Status,0.052533,0.098560,0.016984,0.088699,-0.018705,0.545934,0.155912,-0.003004,1.000000


In [19]:
x=df.drop(['Application_ID','Application_Status'],axis=1)
y=df['Application_Status']

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=43)

In [21]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(x_train,y_train)

LogisticRegression()

In [22]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [23]:
y_pred = log_model.predict(x_test)

In [24]:
accuracy_score(y_pred,y_test)

0.7475728155339806

In [25]:
confusion_matrix(y_test,y_pred)

array([[21, 24],
       [ 2, 56]], dtype=int64)